# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import key_dict
gkey = key_dict['gkey']

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
weather_url = '../expected_output/cities.csv'
weather_data = pd.read_csv(weather_url, index_col=0)
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.8000,-139.0333,80.51,67,100,8.55,PF,1615944433
1,junction city,39.0286,-96.8314,55.00,71,75,13.80,US,1615944433
2,avarua,-21.2078,-159.7750,84.20,74,20,8.05,CK,1615944434
3,los llanos de aridane,28.6585,-17.9182,64.40,63,20,10.36,ES,1615944434
4,solano,38.2333,-121.9511,57.20,47,1,10.36,US,1615944435


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# base url
gmaps.configure(api_key=gkey)
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [5]:
locations = weather_data[['Lat','Lng']].astype(float)
humidity = weather_data['Humidity'].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 max_intensity=humidity.max(),
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
ideal_conditions_df = weather_data[(weather_data['Max Temp'] < 80) & (weather_data['Max Temp'] > 70) & (weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0)]
ideal_conditions_df = ideal_conditions_df.dropna()
ideal_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,mount gambier,-37.8333,140.7667,77.00,44,0,8.05,AU,1615944439
21,busselton,-33.6500,115.3333,72.00,71,0,1.01,AU,1615944441
34,codrington,-38.2667,141.9667,75.70,48,0,4.81,AU,1615944447
40,saint-philippe,-21.3585,55.7679,75.20,64,0,4.61,RE,1615944325
187,riyadh,24.6877,46.7219,77.00,23,0,5.75,SA,1615944248
202,saint-leu,-21.1500,55.2833,75.20,64,0,4.61,RE,1615944512
211,salalah,17.0151,54.0924,73.40,68,0,4.61,OM,1615944515
230,seddon,-37.8068,144.8916,75.99,56,0,3.44,AU,1615944522
231,sao filipe,14.8961,-24.4956,70.83,77,0,3.33,CV,1615944523
239,monywa,22.1167,95.1333,75.18,32,0,5.68,MM,1615944525


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = ideal_conditions_df
hotel_df['Hotel Name'] = ""


In [44]:
hotel_names = []

for index, city in hotel_df.iterrows():
    
    latitude = city['Lat']
    longitude = city['Lng']
    params = {'radius': 5000, 'key':gkey, 'types': 'hotel',
              'location': f'{latitude}, {longitude}'}
    try:
        response = requests.get(base_url, params=params).json()
        hotel_names.append(response['results'][0]['name'])
    except:
        hotel_names.append(np.nan)
        
hotel_df['Hotel Name'] = hotel_names
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,mount gambier,-37.8333,140.7667,77.00,44,0,8.05,AU,1615944439,Mount Gambier
21,busselton,-33.6500,115.3333,72.00,71,0,1.01,AU,1615944441,Busselton
34,codrington,-38.2667,141.9667,75.70,48,0,4.81,AU,1615944447,Yambuk
40,saint-philippe,-21.3585,55.7679,75.20,64,0,4.61,RE,1615944325,Saint-Philippe
187,riyadh,24.6877,46.7219,77.00,23,0,5.75,SA,1615944248,Riyadh
202,saint-leu,-21.1500,55.2833,75.20,64,0,4.61,RE,1615944512,"Saint-Leu, Réunion"
211,salalah,17.0151,54.0924,73.40,68,0,4.61,OM,1615944515,Salalah
230,seddon,-37.8068,144.8916,75.99,56,0,3.44,AU,1615944522,Melbourne
231,sao filipe,14.8961,-24.4956,70.83,77,0,3.33,CV,1615944523,Sao Filipe
239,monywa,22.1167,95.1333,75.18,32,0,5.68,MM,1615944525,Monywa


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
plt.savefig('../expected_output/heatmap_output.png')

<Figure size 432x288 with 0 Axes>